In [1]:
import pandas as pd
import os, glob

from owlready2 import *
import owlready2
print(owlready2.VERSION)


import importlib.util
import sys
spec = importlib.util.spec_from_file_location("rdfutils", "../../../utils/rdfutils.py")
u = importlib.util.module_from_spec(spec)
sys.modules["rdfutils"] = u
spec.loader.exec_module(u)

from datetime import datetime

def NOW():
    now = datetime.now()
    current_time = now.strftime("%H:%M:%S")
    return "Current Time = "+ str(current_time)

%load_ext autoreload
%autoreload 2


0.40


* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation


In [2]:
onto = get_ontology("../pbn_t3_5.owl").load()
comments = u.checkComments(onto)

ID: 0 	 Author: Luc Jonveaux
ID: 1 	 License: CC BY-NC-SA
ID: 2 	 Language: English
ID: 3 	 Project: PROBONO
ID: 4 	 Task: T3.5
ID: 5 	 Description: Creation of a knowledge graph based on a litterature review, augmented by use of LLMs.
ID: 6 	 Changes from: 0.0
ID: 7 	 Changes: Creation of the knowledge graph
ID: 8 	 Changes from: 0.1
ID: 9 	 Changes: Adding relations, simplified ontology, closest items, and descriptions
ID: 10 	 Repository: https://github.com/mm80843/T3.5/
ID: 11 	 TODOs: Quite a number! Creating metaRisks, making it accessible..
ID: 12 	 Creation: 11/11/2023
ID: 13 	 Version: 0.3
ID: 14 	 VersionComment: New blueprints added in 0.3
ID: 15 	 Library: owlready2==0.45


In [3]:
dIDct = u.createDict(onto) 

PBNThing
BenefReturn
Benef
Article
Risk
ISO_Scale
RiskHealth
RiskType
Stakeholder
Stakeholder_Type
Technology
ISO_Impact
ISO_Purpose
Mitigation
BP_Enabler
BP_Transmission
Blueprint
BP_Scale
BP_Phase
BP_Permanent
BP_Type
BP_Intervention
PBNCategory
BP_Theme
BP_Category
aBlueprint


In [4]:
dfR = pd.read_parquet("newrisks.parquet.gzip")
dfR = dfR.replace("","")
dfR

/home/kelu/projets/T3.5/venv/lib/python3.10/site-packages/pandas/core/missing.py:106: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  new_mask[arr_mask] = arr[arr_mask] == x


,Description,Impact,Mitigation,Technologies,People,Owner,src,hash,author,title,doi
0,Zoonotic infections from domestic and wild ani...,Severe economic disruption and disaster if not...,Improved animal-to-human interaction managemen...,"[Dna sequencing for pathogen identification, d...","[General population, healthcare workers, trave...","[Healthcare systems, governments, animal contr...",abideen_mitigation_2020,f5f5af1eef7fe66226065027d070a105,"Abideen, Ahmed Zainul and Mohamad, Fazeeda Bin...",Mitigation strategies to fight the {COVID}-19 ...,10.1108/jhr-04-2020-0109
1,"Spread risk based on population, poverty level...",Potential for localized outbreaks due to mutat...,Improved surveillance and control of environme...,[Data analytics for monitoring population move...,"[General population, travelers, individuals in...","[Healthcare systems, governments, environmenta...",abideen_mitigation_2020,f5f5af1eef7fe66226065027d070a105,"Abideen, Ahmed Zainul and Mohamad, Fazeeda Bin...",Mitigation strategies to fight the {COVID}-19 ...,10.1108/jhr-04-2020-0109
2,"Spread of contagious diseases like influenza, ...","High mortality rates, strain on healthcare sys...","Enhanced hygiene protocols, rapid response pla...","[Vaccines, antiviral drugs, contact tracing apps]","[Global population, healthcare workers, travel...","[Governments, healthcare organizations, intern...",abideen_mitigation_2020,e2fefe64d3f6f552fd8c225341c476cb,"Abideen, Ahmed Zainul and Mohamad, Fazeeda Bin...",Mitigation strategies to fight the {COVID}-19 ...,10.1108/jhr-04-2020-0109
3,"Failure in containing outbreaks, political mis...","Increased spread of disease, misinformation, l...","Transparent communication, effective leadershi...","[Information dissemination tools, crisis manag...","[General population, government officials, med...","[Government, media organizations, public healt...",abideen_mitigation_2020,e2fefe64d3f6f552fd8c225341c476cb,"Abideen, Ahmed Zainul and Mohamad, Fazeeda Bin...",Mitigation strategies to fight the {COVID}-19 ...,10.1108/jhr-04-2020-0109
4,Illicit trade of wildlife meat and potential f...,Increased risk of zoonotic disease transmissio...,"Regulation of wildlife trade, enforcement of f...","[Dna testing for wildlife species, monitoring ...","[Wildlife traders, public health agencies, res...","[Government authorities, wildlife conservation...",abideen_mitigation_2020,e2fefe64d3f6f552fd8c225341c476cb,"Abideen, Ahmed Zainul and Mohamad, Fazeeda Bin...",Mitigation strategies to fight the {COVID}-19 ...,10.1108/jhr-04-2020-0109
...,...,...,...,...,...,...,...,...,...,...,...
18149,Testing alone was not enough to reduce R0 belo...,Inability to reduce R0 below 1 solely through ...,Combination of testing with high-quality infec...,"[Testing technologies, infection control techn...","[General population, healthcare workers]","[Public health authorities, healthcare adminis...",ingram_covid-19_2021,ddc792c4fd824ca02fa7d4432994822d,"Ingram, Carolyn and Downey, Vicky and Roe, Mar...",{COVID}-19 {Prevention} and {Control} {Measure...,10.3390/ijerph18157847
18150,The effectiveness of test and trace depends st...,Ineffective contact tracing can lead to uncont...,Improving coverage and timeliness of contact t...,"[Contact tracing technologies, data management...",[Individuals who require contact tracing],"[Public health authorities, contact tracing or...",ingram_covid-19_2021,ddc792c4fd824ca02fa7d4432994822d,"Ingram, Carolyn and Downey, Vicky and Roe, Mar...",{COVID}-19 {Prevention} and {Control} {Measure...,10.3390/ijerph18157847
18151,Healthcare and nursing home workers testing po...,Increased risk of infection and transmission i...,Implement strict infection prevention and cont...,"[Advanced ppe, rapid testing technologies, and...","[Healthcare workers, nursing home staff, and r...","[Healthcare administrators, facility managers,...",ingram_covid-19_2021,3f594f177c862efe64ddde788d595864,"Ingram, Carolyn and Downey, Vicky and Roe, Mar...",{COVID}-19 {Pr

In [5]:

for ix, row in dfR.iterrows():
    if ix < 20000:
        try:
            newRisk =  u.addItem("Risk", row["Description"], dIDct, onto )
            newRisk.has_RiskName.append(row["Impact"])

            src= u.addItem("Article",row["src"], dIDct, onto )
            newRisk.has_RiskSource.append(src)

            mitig = u.addItem("Mitigation",row["Mitigation"], dIDct, onto )
            newRisk.has_RiskMitigation.append(mitig)
    
            for tech in row["Technologies"]:
                mytech = u.addItem("Technology",tech, dIDct, onto ) 
                newRisk.has_RiskTechnology.append(mytech)
                mytech.has_TechSource.append(src)

            for stakeholder in row["People"]:
                newstakeholder = u.addItem("Stakeholder",stakeholder, dIDct, onto )  
                newRisk.has_RiskSubject.append(newstakeholder)
                                
            for stakeholder in row["Owner"]:
                newstakeholder = u.addItem("Stakeholder",stakeholder, dIDct, onto )   
                newRisk.has_RiskOwner.append(newstakeholder)
        except:
            pass


In [6]:
onto.save("./WIP.owl")

In [7]:
print("Done")

Done
